In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys

project_module_path = os.path.abspath(os.path.join('../'))
if project_module_path not in sys.path:
    print(f'Adding {project_module_path} to system paths')
    sys.path.append(project_module_path)

# %env AWS_PROFILE=development

Adding /Users/bensoorajmohan/Development/ai-playground/pytorch-mnist-play/src to system paths


# Lighter Model v2

## Target
- Aggressively try to reduce the parameters
## Result
- Parameters: 7,316
- Accuracy:
| Accuracy | Best | Final |
| --- | --- | --- |
| Train | 98.83 | 98.83
| Test | 99.39 | 99.26
## Analysis
- Model is light
- No overfitting (train acc < test acc)
- Accuracy has tanked


## Import all the libraries

In [2]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from tqdm import tqdm
from torchvision import datasets, transforms
from torch.utils.tensorboard import SummaryWriter

!pip install torchsummary
from torchsummary import summary

In [3]:
import logging

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

logging.info("logging configured")  # calling the root logger

INFO:root:logging configured


## Configure device

In [4]:
use_cuda = torch.cuda.is_available()
use_mps = torch.backends.mps.is_available()

device: torch.device = None
if use_cuda:
    device = torch.device("cuda")
elif use_mps:
    device = torch.device("mps")
else:
    device = torch.device("cpu")

assert device is not None, "device must not be None"
assert isinstance(device, torch.device), "device must be an instance of torch.device"

## Download the dataset and configure loaders

In [5]:
torch.manual_seed(1)
BATCH_SIZE = 64

# Transforms
train_transforms = transforms.Compose([
    transforms.RandomRotation((-7.0, 7.0), fill=(1,)),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])
test_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train = datasets.MNIST('../../data', train=True, download=True, transform=train_transforms)
test = datasets.MNIST('../../data', train=False, download=True, transform=test_transforms)

dataloader_args = dict(shuffle=True, batch_size=BATCH_SIZE, num_workers=2, pin_memory=True) if (use_cuda or use_mps) else dict(shuffle=True, batch_size=64)

# train dataloader
train_loader = torch.utils.data.DataLoader(train, **dataloader_args)

# test dataloader
test_loader = torch.utils.data.DataLoader(test, **dataloader_args)


## Model summary

In [6]:
from models import ModelV3
summary(ModelV3(), input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             160
              ReLU-2           [-1, 16, 28, 28]               0
            Conv2d-3           [-1, 16, 28, 28]           2,320
              ReLU-4           [-1, 16, 28, 28]               0
       BatchNorm2d-5           [-1, 16, 28, 28]              32
         MaxPool2d-6           [-1, 16, 14, 14]               0
            Conv2d-7           [-1, 10, 14, 14]           1,450
              ReLU-8           [-1, 10, 14, 14]               0
       BatchNorm2d-9           [-1, 10, 14, 14]              20
          Dropout-10           [-1, 10, 14, 14]               0
           Conv2d-11           [-1, 10, 14, 14]             910
             ReLU-12           [-1, 10, 14, 14]               0
      BatchNorm2d-13           [-1, 10, 14, 14]              20
        MaxPool2d-14             [-1, 1

## Train and evaluate the model

### Configuration

In [7]:
from pytorch_mnist_play import Config, Trainer

summaryWriter=SummaryWriter("../../logs/mnist")
tConfig = Config(
    summaryWriter=summaryWriter,
    epochs=15,
    # enable_lr_scheduler=True,
)
print(f"tConfig: {tConfig}")

modelV3 = ModelV3().to(device)

tConfig: batch_size=128 epochs=15 lr=0.01 enable_lr_scheduler=False momentum=0.9 seed=1 log_dir='logs/mnist' summaryWriter=<torch.utils.tensorboard.writer.SummaryWriter object at 0x1111ec090>


### Start the loop

In [8]:
trainer = Trainer(tConfig)
trainer.fit(modelV3, device, train_loader, test_loader, modelV3.name())

Epoch 1: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 938/938 [00:17<00:00, 54.38it/s, batch_id=937, loss=0.1525]
INFO:pytorch_mnist_play.trainer:[TRAIN ModelV3] Epoch 01 - Accuracy: 56259/60000 (93.77%)
INFO:pytorch_mnist_play.trainer:[TEST ModelV3] Epoch 01 - Loss: 0.0483, Accuracy: 9835/10000 (98.35%)
Epoch 2: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 938/938 [00:11<00:00, 81.11it/s, batch_id=937, loss=0.0211]
INFO:pytorch_mnist_play.trainer:[TRAIN ModelV3] Epoch 02 - Accuracy: 58504/60000 (97.51%)
INFO:pytorch_mnist_play.trainer:[TEST ModelV3] Epoch 02 - Loss: 0.0411, Accuracy: 9864/10000 (98.64%)
Epoch 3: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████